In [1]:
### Carga de paquetes y librerias
#!pip install keras-tuner
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf #Framework para deep learning
from tensorflow import keras #API que contiene la mayoría de funciones para las RN
from keras_tuner.tuners import RandomSearch

keras.utils.set_random_seed(812)

2024-05-20 19:25:01.206437: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
### Carga de datos fasion_mnist
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

### Estandarización de los datos
x_train2 = x_train/255 # Valores entre 0 y 1
x_test2 = x_test/255

# Definicion del modelo para Tunear

In [9]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))

    ## Definición de la primera Capa Oculta con ajute de hiperparametros
    ## Elegir numero optimo de neuronas
    hp_unit_1 = hp.Int('unit_1', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_unit_1, activation='relu'))

    ## Definición de capa de dropout
    ## Elegir el dropout rate optimo - Regularizacion
    hp_dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.3, step=0.1)
    model.add(keras.layers.Dropout(hp_dropout_rate))

    ## Definición de segunda capa oculta con ajuste de hiperparametros
    ## Elegir funcion de activacion optima
    hp_activation_2 = hp.Choice('activation', values=['relu', 'tanh'])
    hp_unit_2 = hp.Int('unit_2', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_unit_2, activation=hp_activation_2))

    ## Definicion de la capa de salida
    model.add(keras.layers.Dense(10, activation='softmax'))

    ## Definicion de la tasa de aprendizaje (learning_rate) del optimizador
    ## Elegir un valor optimo entre 0.1, 0.01, 0.001, 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[0.1, 0.01, 0.001, 0.0001])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [10]:
### Definición del Tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=2,
    directory='results_tuner',
    project_name='Demo'
)

In [12]:
## Ejecutar tuning de hiperparametros
tuner.search(x_train2, y_train, epochs=5, validation_data=(x_test2, y_test))

Trial 5 Complete [00h 01m 36s]
val_accuracy: 0.14910000190138817

Best val_accuracy So Far: 0.8814999759197235
Total elapsed time: 00h 07m 13s


In [15]:
tuner.get_best_hyperparameters()[0].get('activation')

'relu'

In [19]:
tuner.get_best_hyperparameters()[0].get('dropout_rate')

0.1

In [17]:
### Mostrar el mejor modelo
for h_param in [f"unit_{i}" for i in range(1,3)] + ['activation'] + ['learning_rate']:
    print(h_param, tuner.get_best_hyperparameters()[0].get(h_param))

unit_1 320
unit_2 448
activation relu
learning_rate 0.001


In [18]:
### Almacenar el mejor modelo
best_model = tuner.get_best_models()[0]
### Definir la arquitectura del modelo según hiperparámetros optimos
best_model.build(x_train2.shape)
### Resumen de la arquitectura
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 320)               251200    
                                                                 
 dropout (Dropout)           (None, 320)               0         
                                                                 
 dense_1 (Dense)             (None, 448)               143808    
                                                                 
 dense_2 (Dense)             (None, 10)                4490      
                                                                 
Total params: 399,498
Trainable params: 399,498
Non-trainable params: 0
_________________________________________________________________


In [20]:
### Ajuste de la red neuronal con hiperparámetros optimos
history = best_model.fit(x_train2, y_train, epochs=10, validation_data = (x_test2, y_test))

### Cual es el epoch con max val_accuracy
val_accuracy_per_epoch = history.history['val_accuracy']
best_epoch = val_accuracy_per_epoch.index(max(val_accuracy_per_epoch))+1
print(f"Best epoch: {best_epoch}")

Epoch 1/10
1875/1875 [==============================] - 11s 5ms/step - loss: 0.2864 - accuracy: 0.8929 - val_loss: 0.3408 - val_accuracy: 0.8787
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2757 - accuracy: 0.8963 - val_loss: 0.3491 - val_accuracy: 0.8788
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2656 - accuracy: 0.8997 - val_loss: 0.3299 - val_accuracy: 0.8843
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2552 - accuracy: 0.9026 - val_loss: 0.3279 - val_accuracy: 0.8866
Epoch 5/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2465 - accuracy: 0.9074 - val_loss: 0.3430 - val_accuracy: 0.8824
Epoch 6/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2435 - accuracy: 0.9086 - val_loss: 0.3413 - val_accuracy: 0.8866
Epoch 7/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2352 - accuracy: 0.9099 - val_loss: 0.3386 - val_accurac

In [23]:
### Utilizar el metodo evaluate para evaluar la red neuronal
test_loss, test_accuracy = best_model.evaluate(x_test2, y_test)
print('Test accuracy: ', test_accuracy)

313/313 [==============================] - 1s 2ms/step - loss: 0.3225 - accuracy: 0.8929
Test accuracy:  0.8928999900817871
